In [3]:
PROJECT = "felipe-sandbox-354619"
DATASET = "financial_fraud"
TABLE = "transactions"
MODEL = ""
BUCKET="gs://" + PROJECT_ID + "-bucket"

In [ ]:
# initialize clients
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

In [20]:
query = """SELECT * FROM `{0}.{1}.{2}` 
LIMIT 10""".format(PROJECT,DATASET, TABLE)

job = client.query(query)
job.to_dataframe()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,256,DEBIT,1567.83,C746082020,0.00,0.00,C703610675,740604.98,742172.81,0,0
1,256,DEBIT,13808.95,C1507261707,64202.00,50393.05,C63420742,127123.26,140932.22,0,0
2,256,DEBIT,2365.81,C1962961815,80414.00,78048.19,C207766181,28846.20,31212.01,0,0
3,256,DEBIT,857.47,C1265211074,228517.00,227659.53,C481841252,36278.09,37135.55,0,0
4,256,DEBIT,1564.55,C2116094677,74492.48,72927.93,C1598593494,2013322.78,2014887.33,0,0
5,256,DEBIT,11049.55,C258685558,9220.00,0.00,C1767078524,43188.84,54238.39,0,0
6,256,DEBIT,2704.30,C398991382,1627.73,0.00,C1702465296,2340019.02,2342723.32,0,0
7,256,DEBIT,13881.23,C1797924681,20755.00,6873.77,C538563383,755326.02,769207.25,0,0
8,256,DEBIT,1864.06,C798347241,0.00,0.00,C1213145652,204816.19,206680.25,0,0
9,256,DEBIT,6603.91,C1664015391,0.00,0.00,C1967864415,125051.35,131655.27,0,0


In [ ]:
query = """CREATE OR REPLACE VIEW `{0}.{1}.{2}_clean` AS
SELECT type, amount, oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest, isFraud 
FROM `{0}.{1}.{2}`""".format(PROJECT,DATASET, TABLE)

job = client.query(query)
job.to_dataframe()

""


In [21]:
query = """SELECT * FROM `{0}.{1}.{2}_clean` 
LIMIT 10""".format(PROJECT,DATASET, TABLE)

job = client.query(query)
job.to_dataframe()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,DEBIT,668.03,10157.0,9488.97,761313.06,761981.08,0
1,DEBIT,947.89,0.0,0.00,3829977.34,3830925.24,0
2,DEBIT,16266.06,1146.0,0.00,50858.83,67124.89,0
3,DEBIT,1225.71,0.0,0.00,372741.05,373966.76,0
4,DEBIT,9300.54,800.0,0.00,499739.31,509039.85,0
5,DEBIT,2545.97,30193.0,27647.03,1148758.43,1151304.40,0
6,DEBIT,21070.42,7562.0,0.00,770346.13,791416.56,0
7,DEBIT,11796.85,69595.0,57798.15,98199.18,109996.03,0
8,DEBIT,16837.01,36319.0,19481.99,59361.10,76198.11,0
9,DEBIT,28870.59,61267.0,32396.41,3339833.85,3565806.04,0


In [22]:
query = """CREATE OR REPLACE MODEL
`{0}.{1}.{2}_logreg_model`
OPTIONS(MODEL_TYPE='LOGISTIC_REG', INPUT_LABEL_COLS=['isFraud'])
AS
SELECT * FROM `{0}.{1}.{2}_clean`""".format(PROJECT,DATASET, TABLE)

job = client.query(query)
job.to_dataframe()

""


In [23]:
query = """SELECT * FROM
  ML.EVALUATE(MODEL `{0}.{1}.{2}_logreg_model`)""".format(PROJECT,DATASET, TABLE)

job = client.query(query)
job.to_dataframe()

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.998884,0.0,0.008085,0.963


In [27]:
query = """SELECT * FROM
  ML.PREDICT(MODEL `{0}.{1}.{2}_logreg_model`,
            (SELECT * FROM `{0}.{1}.{2}_clean` LIMIT 10))""".format(PROJECT,DATASET, TABLE)

job = client.query(query)
job.to_dataframe()

,predicted_isFraud,predicted_isFraud_probs,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,0,"[{'label': 1, 'prob': 0.0009906192683477619}, ...",DEBIT,3919.33,87955.31,84035.98,7796862.27,7800781.60,0
1,0,"[{'label': 1, 'prob': 0.001837498130628047}, {...",DEBIT,21411.96,105874.00,84462.04,268253.60,289665.57,0
2,0,"[{'label': 1, 'prob': 0.0017858839894579035}, ...",DEBIT,3186.31,43027.00,39840.69,565746.28,568932.59,0
3,0,"[{'label': 1, 'prob': 0.0015085100654674856}, ...",DEBIT,2463.89,0.00,0.00,2623663.95,2626127.83,0
4,0,"[{'label': 1, 'prob': 0.0017153741896128879}, ...",DEBIT,4241.32,32464.00,28222.68,1059625.36,1063866.68,0
5,0,"[{'label': 1, 'prob': 0.0011237238995329398}, ...",DEBIT,3060.25,80297.95,77237.70,6249298.32,6252358.56,0
6,0,"[{'label': 1, 'prob': 0.0007877305849839296}, ...",DEBIT,15233.66,61482.00,46248.34,10622361.49,10637595.16,0
7,0,"[{'label': 1, 'prob': 0.0016026329353811364}, ...",DEBIT,3414.12,72060.16,68646.04,1898979.01,1902393.13,0
8,0,"[{'label': 1, 'prob': 0.0006206751051490456}, ...",DEBIT,1168.26,0.00,0.00,13497907.01,13499075.27,0
9,0,"[{'label': 1, 'prob': 0.0012689192502917258}, ...",DEBIT,1314.08,0.00,0.00,4740567.59,4741881.66,0
